# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [ ]:
DATA_FOLDER = '' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [3]:
import glob
import pandas as pd
import numpy as np
import calendar
from datetime import datetime
pd.options.mode.chained_assignment = None 

# Assumptions for the analysis 
# We assume that the data files are complete and that data given is cumulative over the intermediate days

# Load Guinea data

path =r'Data/ebola/guinea_data' 
allFiles = glob.glob(path + "/*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_, header=0)
    list_.append(df)
frame_guinea = pd.concat(list_)

frame_guinea = frame_guinea[['Description','Date','Totals']]

# We first check for unique labels in the 'Description'-column 
frame_guinea.Description.unique()

# We select only the registered new cases as well as registered deaths 

frame_guinea = frame_guinea.loc[frame_guinea['Description'] .isin(['Total new cases registered so far', 'New deaths registered', 'New deaths registered today'])]

frame_guinea['Country'] = 'Guinea'

# Load Liberia data 

path =r'Data/ebola/liberia_data' 
allFiles = glob.glob(path + "/*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_, header=0)
    list_.append(df)
frame_liberia = pd.concat(list_, ignore_index=True)

frame_liberia = frame_liberia[['Variable','Date','National']]

frame_liberia.columns = ['Description','Date','Totals']

# Checking for unique labels in the 'Description'-column 
frame_liberia.Description.unique()

frame_liberia = frame_liberia.loc[frame_liberia['Description'].isin(['Newly reported deaths', 'New case/s (confirmed)'])]

frame_liberia['Country'] = 'Liberia'

# Load Sierra Leone data 

path =r'Data/ebola/sl_data' 
allFiles = glob.glob(path + "/*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_, header=0)
    list_.append(df)
frame_sierraleone = pd.concat(list_)

frame_sierraleone = frame_sierraleone[['variable','date','National']]

frame_sierraleone.columns = ['Description','Date','Totals']

# Checking for unique labels in the 'Description'-column 
frame_sierraleone.Description.unique()

frame_sierraleone = frame_sierraleone.loc[frame_sierraleone['Description'].isin(['new_confirmed','death_confirmed'])]

frame_sierraleone['Country'] = 'Sierraleone'

# Create one Dataframe

list_ = []
list_.append(frame_guinea)
list_.append(frame_liberia)
list_.append(frame_sierraleone)

frame_total = pd.concat(list_, ignore_index=True)

# Convert the Date column into datetime type

frame_total.Date = pd.to_datetime(frame_total.Date)

# Create average deaths and average new cases 
# Since we assumed that the data is complete, we summed up all new cases from each data set and divided by 30 days as 
# an average for a month
# We did the same for deaths
# For simplicity the months we no data have a 0 average

info = pd.DataFrame(columns=('Country', 'Month', 'Av. deaths', 'Av. new cases'))

for i in range(6,13):
    info.loc[(i-5)*3-2] = ['Guinea', i, pd.to_numeric(frame_total[(frame_total.Description.isin(['New deaths registered', 'New deaths registered today'])) & (frame_total['Date'].dt.month == i)].Totals, errors='coerce').sum()/calendar.monthrange(2014,i)[1], 
                     pd.to_numeric(frame_total[(frame_total.Description.isin(['Total new cases registered so far'])) & (frame_total['Date'].dt.month == i)].Totals, errors='coerce').sum()/calendar.monthrange(2014,i)[1]]
    info.loc[(i-5)*3-1] = ['Liberia', i, pd.to_numeric(frame_total[(frame_total.Description.isin(['Newly reported deaths'])) & (frame_total['Date'].dt.month == i)].Totals, errors='coerce').sum()/calendar.monthrange(2014,i)[1], 
                     pd.to_numeric(frame_total[(frame_total.Description.isin(['New case/s (confirmed)'])) & (frame_total['Date'].dt.month == i)].Totals, errors='coerce').sum()/calendar.monthrange(2014,i)[1]]
    info.loc[(i-5)*3] = ['Sierra Leone', i, pd.to_numeric(frame_total[(frame_total.Description.isin(['new_deaths'])) & (frame_total['Date'].dt.month == i)].Totals, errors='coerce').sum()/calendar.monthrange(2014,i)[1], 
                     pd.to_numeric(frame_total[(frame_total.Description.isin(['new_confirmed'])) & (frame_total['Date'].dt.month == i)].Totals, errors='coerce').sum()/calendar.monthrange(2014,i)[1]]

info

ValueError: No objects to concatenate

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [2]:
import glob
import os
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None 

path = r'DataHW1/microbiome' 
#Load metadata file into a data frame
metadata = pd.read_excel("DataHW1/microbiome/metadata.xls")

#Load all files (includes metadata file again!)
all_files = glob.glob(os.path.join(path, "*.xls")) 
#Drop first entry from list as list starts with metadata.xls 
del all_files[0]


microbiome = pd.DataFrame()
dflist_ = [] #list for data frames
N=0 #counter to identify row in metadata 
for file_ in all_files:
    frame = pd.read_excel(file_, header=None)       #Read current file into the df frame
    filename = os.path.basename(file_) #readout file name
    filename_short = filename[:-4]     #drop file extension
    frame['Filename'] = filename_short #Add original filename in a new column, as a check
    frame['Group'] = metadata.loc[N,"GROUP"]
    frame['Sample'] = metadata.loc[N,"SAMPLE"]
    N= N+1 
    dflist_.append(frame)
     
microbiome = pd.concat(dflist_, ignore_index=True)   #using  ignore_index=True option to provide unique index for each individual entry
#Checking uniqueness of current index
microbiome.index.is_unique
#Returned True
#Otherwise could consider merging several columns into an index as done in tutorial baseball example

#Replacing NAN in SAMPLE for MID1 with "unknown"
microbiome['Sample'] = microbiome['Sample'].replace(np.nan, 'unknown', regex=True)
#Checking that all NaN where captured
microbiome.isnull().values.any()
#Returned False

#Relabel first 2 columns
microbiome.columns = ['Species', 'Quantity', 'Filename', 'Group', 'Sample']
microbiome.Species

#Following FAQ 19 we construct meaningful index by combining 'Species', 'Group' and 'Sample'
new_id = microbiome.Species + microbiome.Group + microbiome.Sample
microbiome_newind = microbiome.copy()
microbiome_newind.index = new_id
microbiome_newind.head()
#Checking uniqueness of current index
microbiome_newind.index.is_unique
#Returned True
microbiome_newind

,Species,Quantity,Filename,Group,Sample
"Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Desulfurococcaceae IgnisphaeraEXTRACTION CONTROLunknown","Archaea ""Crenarchaeota"" Thermoprotei Desulfuro...",7,MID1,EXTRACTION CONTROL,unknown
"Archaea ""Crenarchaeota"" Thermoprotei Desulfurococcales Pyrodictiaceae PyrolobusEXTRACTION CONTROLunknown","Archaea ""Crenarchaeota"" Thermoprotei Desulfuro...",2,MID1,EXTRACTION CONTROL,unknown
"Archaea ""Crenarchaeota"" Thermoprotei Sulfolobales Sulfolobaceae StygiolobusEXTRACTION CONTROLunknown","Archaea ""Crenarchaeota"" Thermoprotei Sulfoloba...",3,MID1,EXTRACTION CONTROL,unknown
"Archaea ""Crenarchaeota"" Thermoprotei Thermoproteales Thermofilaceae ThermofilumEXTRACTION CONTROLunknown","Archaea ""Crenarchaeota"" Thermoprotei Thermopro...",3,MID1,EXTRACTION CONTROL,unknown
"Archaea ""Euryarchaeota"" ""Methanomicrobia"" Methanocellales Methanocellaceae MethanocellaEXTRACTION CONTROLunknown","Archaea ""Euryarchaeota"" ""Methanomicrobia"" Meth...",7,MID1,EXTRACTION CONTROL,unknown
"Archaea ""Euryarchaeota"" ""Methanomicrobia"" Methanosarcinales Methanosarcinaceae MethanimicrococcusEXTRACTION CONTROLunknown","Archaea ""Euryarchaeota"" ""Methanomicrobia"" Meth...",1,MID1,EXTRACTION CONTROL,unknown
"Archaea ""Euryarchaeota"" ""Methanomicrobia"" Methanosarcinales Methermicoccaceae MethermicoccusEXTRACTION CONTROLunknown","Archaea ""Euryarchaeota"" ""Methanomicrobia"" Meth...",1,MID1,EXTRACTION CONTROL,unknown
"Archaea ""Euryarchaeota"" Archaeoglobi Archaeoglobales Archaeoglobaceae FerroglobusEXTRACTION CONTROLunknown","Archaea ""Euryarchaeota"" Archaeoglobi Archaeogl...",1,MID1,EXTRACTION CONTROL,unknown
"Archaea ""Euryarchaeota"" Archaeoglobi Archaeoglobales Archaeoglobaceae GeoglobusEXTRACTION CONTROLunknown","Archaea ""Euryarchaeota"" Archaeoglobi Archaeogl...",1,MID1,EXTRACTION CONTROL,unknown
"Archaea ""Euryarchaeota"" Halobacteria Halobacteriales Halobacteriaceae HaloplanusEXTRACTION CONTROLunknown","Archaea ""Euryarchaeota"" Halobacteria Halobacte...",1,MID1,EXTRACTION CONTROL,unknown


## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [4]:
# from IPython.core.display import HTML
# HTML(DataHW1/titanic.html')
import glob
import os
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None 

path = r'DataHW1' 
#Load titanic xls file into a data frame
titanic = pd.read_excel("DataHW1/titanic.xls")

# list of the attributes and their type and range:
#
# pclass: Integer with 3 options: 1,2,3
# survived: Integer with 2 options: 1 (survived), 0 (died)
# name: object (string)
# sex: Object with 2 options: F, M
# age: float 64 range=0 to 80
# sibsp: Integer with multiple options: [0, 1, 2, 3, 4, 5, 8]
# parch: Integer with multiple options: [0, 2, 1, 4, 3, 5, 6, 9]
# ticket: object
# fare: float64 range 0 to 263
# cabin: object combined of 1 letter and a number (finite range)
# embarked: object with 3 options: C(herbourg), Q(ueenstown), S(outhampton)
# boat: object (as label is string)
# body: float64 range=1 to 328
# homedest: object

# We will treat the following variables as categorical data: pclass, survived, sex, embarked
pd.Categorical(titanic.pclass)
pd.Categorical(titanic.survived)
pd.Categorical(titanic.sex)
pd.Categorical(titanic.embarked)
# Cabins and boats are finite sets with clear labels and hence could be considered as categorical data (appear to be treated as such on the webpage)
# However in both cases there are examples with double entries (e.g. 'C22 C26'), so one can not categorise cabin them separaef
# Note one could consider the cabin class (A, B, C, D or E) as a categorical variable.
# Boats has the additional issue that people who died are labelled as NaN
titanic


,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
5,1,1,"Anderson, Mr. Harry",male,48.0000,0,0,19952,26.5500,E12,S,3,NaN,"New York, NY"
6,1,1,"Andrews, Miss. Kornelia Theodosia",female,63.0000,1,0,13502,77.9583,D7,S,10,NaN,"Hudson, NY"
7,1,0,"Andrews, Mr. Thomas Jr",male,39.0000,0,0,112050,0.0000,A36,S,NaN,NaN,"Belfast, NI"
8,1,1,"Appleton, Mrs. Edward Dale (Charlotte Lamson)",female,53.0000,2,0,11769,51.4792,C101,S,D,NaN,"Bayside, Queens, NY"
9,1,0,"Artagaveytia, Mr. Ramon",male,71.0000,0,0,PC 17609,49.5042,NaN,C,NaN,22.0,"Montevideo, Uruguay"


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#Task 2: Plot histograms for the travel class, embarkation port, sex and age attributes. For the latter one, use discrete decade intervals.
mb = pd.read_excel('Data_copy/titanic.xls')
#mb = mb.replace(np.nan, 'Unknown', regex=True) # How do you plot a string in hist
#age = pd.cut(mb.age, [0,10,20,30,40,50,60,70,80,90,100,110])

treatment_map = {'S': 0, 'C': 1, 'Q': 2}
embarked_tonum = mb.embarked.map(treatment_map)

treatment_map1 = {'male': 0, 'female': 1}
sex_tonum = mb.sex.map(treatment_map1)

mb.pclass.hist()
plt.show()
embarked_tonum.hist()
plt.show()
sex_tonum.hist()
plt.show()
age_interval = np.array([0,10,20,30,40,50,60,70,80,90,100,110])
mb.age.hist(bins=age_interval)
plt.show()
mb.embarked.value_counts()

In [ ]:
#Task3 Calculate the proportion of passengers by cabin floor. Present your results in a pie chart.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

 
mb = mb.replace(np.nan, 'Unknown', regex=True)
floor = []
for i in range(len(mb)):
    floor.append(mb.cabin[i][0])

data_floor = pd.DataFrame({'value':[floor.count('A'), floor.count('B'), floor.count('C'), floor.count('D'), floor.count('E'), floor.count('F'), floor.count('U')],
                    'Floor':['A', 'B', 'C', 'D', 'E', 'F', 'U']})   
data_floor.value.plot.pie(labels=data_floor.Floor,autopct='%0.2f', fontsize=10, figsize=(15, 15))
#patches, texts = plt.pie(sizes, colors=colors, shadow=True, startangle=90)
plt.legend(data_floor.Floor, loc="best",)
plt.axis('equal')
plt.tight_layout()
plt.show()

#counts_floor = { 'A':floor.count('A'), 
#                 'B': floor.count('B'), 
#                  'C': floor.count('C'),
#                 'D': floor.count('D'),
#                'E':floor.count('E'),
#                'F':floor.count('F'),
#                'U':floor.count('U')}
#pd.Series(counts_floor)

 
# Data to plot
#labels = 'A', 'B', 'C', 'D', 'E', 'F', 'Unknown'
#sizes = [floor.count('A'), floor.count('B'), floor.count('C'), floor.count('D'), floor.count('E'), floor.count('F'), floor.count('U')]
#colors = ['red', 'yellowgreen', 'lightcoral', 'lightskyblue', 'blue', 'green', 'gold']
#explode = (0.1, 0, 0, 0)  # explode 1st slice
 
# Plot
#plt.pie(sizes, labels=labels, colors=colors,  autopct='%1.1f%%',  shadow=True)
#plt.axis('equal')
#plt.show()
#plt.tight_layout()



In [ ]:
#Task 4:For each travel class, calculate the proportion of the passengers that survived. Present your results in pie charts.
# 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

mb = pd.read_excel('Data_copy/titanic.xls')
mb = mb.replace(np.nan, 'Unknown', regex=True)

#for i in range(len(mb)):
class1 = mb.loc[mb['pclass'] == 1]
class1['survived'].value_counts().plot.pie(autopct='%0.2f', fontsize=10, figsize=(15, 15))
plt.show()
plt.legend(data_floor.Floor, loc="best",)
plt.axis('equal')
plt.tight_layout()

class2 = mb.loc[mb['pclass'] == 2]
class2['survived'].value_counts().plot.pie(autopct='%0.2f', fontsize=10, figsize=(15, 15))
plt.show()
plt.legend(data_floor.Floor, loc="best",)
plt.axis('equal')
plt.tight_layout()

class3 = mb.loc[mb['pclass'] == 3]
class3['survived'].value_counts().plot.pie(autopct='%0.2f', fontsize=10, figsize=(15, 15))
plt.show()
plt.legend(data_floor.Floor, loc="best",)
plt.axis('equal')
plt.tight_layout()

In [ ]:
#Task5
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

mb = pd.read_excel('DataHW1/titanic.xls')
mb = mb.replace(np.nan, 'Unknown', regex=True)

#for i in range(len(mb)):
survived = mb.loc[mb['survived'] == 1]
not_survived =  mb.loc[mb['survived'] == 0]
class_sex_survived = survived.pclass.astype(str) + survived.sex
class_sex_not_survived = not_survived.pclass.astype(str) + not_survived.sex

count_class_sex_not_survived = class_sex_not_survived.value_counts()
count_class_sex_survived = class_sex_survived.value_counts()

#classsexsurvived = pd.to_numeric(classsexsurvived,errors='raise')
treatment_map = {'1male': 0, '1female': 1, '2male': 2,'2female': 3, '3male': 4,'3female':5}
class_sex_survived = class_sex_survived.map(treatment_map)
class_sex_not_survived = class_sex_not_survived.map(treatment_map)
pd.Series([632, 1638, 569, 115], 
    index=['Firmicutes', 'Proteobacteria', 'Actinobacteria', 'Bacteroidetes'])

count_class_sex_not_survived.sort_index()
count_class_sex_survived.sort_index()
porportion_class_sex = count_class_sex_survived.divide(count_class_sex_not_survived + count_class_sex_survived)
porportion_class_sex.plot.bar()
#plt.axis('equal')
plt.show()
porportion_class_sex
#count = classsexsurvived.value_counts()
#count
#survived
#classsexsurvived = []

    #.append(survived.pclass, ignore_index=True)
    #class_sex_survived.append(survived.sex[i][0])

#class1 =survived.loc[mb['pclass'] == 1]
#SexM_class1 =class1.loc[mb['sex'] == male]
#SexF_class1 =class1.loc[mb['sex'] == female]







In [ ]:
#Task6: Create 2 equally populated age categories and calculate survival proportions by age category, 
#travel class and sex. Present your results in a DataFrame with unique index.
import pandas as pd
import numpy as np

mb = pd.read_excel('Data_copy/titanic.xls')
#mb = mb.replace(np.nan, 'Unknown', regex=True)
survived = mb.loc[mb['survived'] == 1]

#Take away all null values
survived = survived[survived.age.notnull()]

survived['age_categories'] = pd.qcut(survived.age, 2)

classsexsurvived = survived.age_categories.astype(str) + survived.pclass.astype(str) + survived.sex
value_classsexsurvived = classsexsurvived.value_counts()
value_classsexsurvived = value_classsexsurvived.divide(value_classsexsurvived.sum()).to_frame()
value_classsexsurvived.rename(columns={0:'Proportions'}, inplace=True)
value_classsexsurvived
value_classsexsurvived.index.is_unique
value_classsexsurvived

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here